## MatlantisSSHServiceの検証


### 接続

In [ ]:
import os
from dotenv import load_dotenv
from matlantis_ssh_service import MatlantisSSHService

load_dotenv()

# ---接続
mss = MatlantisSSHService()
mss.connect(os.getenv("WEBSOCAT_BIN"),
            os.getenv("MATLANTIS_DOMAIN"),
            os.getenv("MATLANTIS_USER_ID"),
            os.getenv("NOTEBOOK_PRE_SHARED_KEY"),
            os.getenv("USER_NAME"),
            os.getenv("IDENTITY_FILE"),)

# ---ためしにコマンド実行
result = mss._execute_command("ls")
print(result.stdout)

In [ ]:
# ---ディレクトリのアップロード
mss.upload_directory("./test_dir/", "~/test_dir")

result = mss._execute_command("ls")
print(result.stdout)

In [ ]:
# ---ディレクトリのダウンロード
mss.download_directory("~/test_dir2", "./test_dir2")


In [ ]:
mss.disconnect()

## 模索場

In [ ]:
import paramiko
import os

ssh_config = paramiko.SSHConfig()
config_path = os.path.expanduser('~/.ssh/config')

with open(config_path, 'r') as f:
    ssh_config.parse(f)

host = 'matlantis'

config = ssh_config.lookup(host)
config

In [ ]:
config['proxycommand']

In [ ]:
f"cmd.exe /c {config['proxycommand']}"

In [ ]:
for key in ['hostname', 'user', 'proxycommand', 'identityfile']:
    if config.get(key) is None:
        assert False, f"{key} is not found in config"


proxy = paramiko.ProxyCommand(f"cmd.exe /c {config['proxycommand']}")

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())


ssh.connect(
    hostname=config['hostname'],
    username=config['user'],
    port=config.get('port', 22),
    key_filename=config['identityfile'][0] if isinstance(config['identityfile'], list) else config['identityfile'],
    sock=proxy
)


In [ ]:
import subprocess
import time
import os
from dotenv import load_dotenv
from fabric import Connection

load_dotenv()

WEBSOCAT_BIN = r"websocat.x86_64-pc-windows-gnu.exe"
if not WEBSOCAT_BIN:
    raise RuntimeError("websocat が PATH に見つかりません。インストールしてください。")

# WebSocket エンドポイント（例）

MATLANTIS_DOMAIN = os.getenv("MATLANTIS_DOMAIN")
MATLANTIS_USER_ID = os.getenv("MATLANTIS_USER_ID")
NOTEBOOK_PRE_SHARED_KEY = os.getenv("NOTEBOOK_PRE_SHARED_KEY")
USER_NAME = os.getenv("USER_NAME")
IDENTITY_FILE = os.getenv("IDENTITY_FILE")

ws_url = f"wss://{MATLANTIS_DOMAIN}/nb/{MATLANTIS_USER_ID}/default/api/ssh-over-ws"
local_port = 2222

# websocat をローカル TCP リスン -> remote ws に接続する形で起動
proc = subprocess.Popen([
    WEBSOCAT_BIN,
    "--binary",
    f'-H=cookie: matlantis-notebook-pre-shared-key={NOTEBOOK_PRE_SHARED_KEY}',
    f"tcp-l:0.0.0.0:{local_port}",  # 接続をローカルで待つ
    ws_url
])

# 少し待ってポートが立ち上がるのを待つ（単純化）
time.sleep(0.5)

try:
    # Fabric で接続（ローカルポートに接続するだけ）
    conn = Connection(
        host="127.0.0.1",
        user=USER_NAME,
        port=local_port,
        connect_kwargs={
            "key_filename": IDENTITY_FILE
        }
    )
    print("実行待ち...")
    result = conn.run("ls", hide=True)
    print("実行完了!")
    print(result.stdout)
    conn.close()
finally:
    proc.terminate()
    proc.wait()